In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from nlppln import WorkflowGenerator
cwl_working_dir = '/home/dafne/cwl-working-dir/'
#cwl_working_dir = '/home/jvdzwaan/cwl-working-dir/'

In [3]:
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    print(wf.list_steps())

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-11 11:02:31.072061. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


Steps
  SafarAnalyze............. out_files = wf.SafarAnalyze(cp, in_files[, analyzer, xmx])
  SafarStem................ out_files = wf.SafarStem(cp, in_files[, stemmer])
  align.................... changes, metadata = wf.align(file1, file2[, out_dir])
  apachetika............... out_files = wf.apachetika(in_files[, tika_server])
  archive2dir.............. out_dir = wf.archive2dir(archive[, remove-dir-structure])
  basic-text-statistics.... metadata_out = wf.basic_text_statistics(in_files, out_file)
  check-utf8............... utf8_files = wf.check_utf8(in_dir[, convert])
  clear-xml-elements....... out_file = wf.clear_xml_elements(element, xml_file)
  copy-and-rename.......... copy = wf.copy_and_rename(in_file[, rename])
  create-chunked-list...... chunks = wf.create_chunked_list(in_dir[, out_name])
  delete-empty-files....... out_files = wf.delete_empty_files(in_dir)
  extract_metadata......... out_meta, out_txt = wf.extract_metadata(in_file)
  filter-nes............... filtered_ner

In [4]:
# remove openiti metadata and divide a file in books/chapters 

with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    #print(wf.list_steps())
    
    txt_file = wf.add_input(txt_file='File')
    
    txt_file = wf.openiti2txt(in_file=txt_file)
    chapters = wf.split_text_openiti_markers(in_file=txt_file)
    snippets = wf.split_text_size(in_file=chapters, scatter='in_file', scatter_method='dotproduct')
    
    out_files = wf.flatten_list(list=snippets)
    
    wf.add_outputs(out_files=out_files)
    
    wf.save('../adhtools/cwl/split-file-chapters.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-09 11:55:47.449271. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [5]:
# split books/chapters for a directory of text files
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    #print(wf.list_steps())
    
    in_dir = wf.add_input(in_dir='Directory')
    
    txt_files = wf.ls(in_dir=in_dir)
    chapters = wf.split_file_chapters(txt_file=txt_files,
                                      scatter='txt_file', scatter_method='dotproduct')
    
    wf.add_outputs(texts=chapters)
    
    wf.save('../adhtools/cwl/split-dir-chapters.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-09 11:55:55.779504. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [6]:
# analyze file (first split it into multiple smaller subfiles based on regexes)
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    
    analyzer = wf.add_input(analyzer='enum', symbols=['Alkhalil', 'BAMA'], default='Alkhalil')
    txt_file = wf.add_input(txt_file='File')
    metadata = wf.add_input(metadata='File')
    cp = wf.add_input(cp='string')
    split_regex_small = wf.add_input(split_regex_small='string[]', default=['Milestone300', '### |', '### ||'])
        
    txt_file = wf.openiti2txt(in_file=txt_file)
    snippets = wf.split_text(in_file=txt_file, regex=split_regex_small)
        
    analyzed_files = wf.SafarAnalyze(in_files=snippets, analyzer=analyzer, cp=cp)
    merged_file = wf.merge_safar_xml(in_files=analyzed_files)
    #filtered_file = wf.safar_filter_analyses(in_file=merged_file)
    
    out_file = wf.safar_add_metadata_file(in_file=merged_file, in_file_meta=metadata)
    
    # Output is one xml file
    wf.add_outputs(out_file=out_file)
    
    wf.save('../adhtools/cwl/safar-split-and-analyze-file-no-filtering.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-09 11:55:58.711092. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [7]:
# analyze file (first split it into multiple smaller subfiles based on file size)
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    
    analyzer = wf.add_input(analyzer='enum', symbols=['Alkhalil', 'BAMA'], default='Alkhalil')
    txt_file = wf.add_input(txt_file='File')
    metadata = wf.add_input(metadata='File')
    cp = wf.add_input(cp='string')
    size = wf.add_input(size='int?')
        
    txt_file = wf.openiti2txt(in_file=txt_file)
    snippets = wf.split_text_size(in_file=txt_file, size=size)
        
    analyzed_files = wf.SafarAnalyze(in_files=snippets, analyzer=analyzer, cp=cp)
    merged_file = wf.merge_safar_xml(in_files=analyzed_files)
    filtered_file = wf.safar_filter_analyses(in_file=merged_file)
    
    out_file = wf.safar_add_metadata_file(in_file=filtered_file, in_file_meta=metadata)
    
    # Output is one xml file
    wf.add_outputs(out_file=out_file)
    
    wf.save('../adhtools/cwl/safar-split-and-analyze-file.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-09 11:56:01.920081. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [8]:
# analyze file 
# - remove openiti metadata
# - divide in chapters (separate headings)
# - split based on file size
# - analyze
# - merge xml files
# - filter analyses
# - add metadata
# Result: an xml file for a book
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    
    analyzer = wf.add_input(analyzer='enum', symbols=['Alkhalil', 'BAMA'], default='Alkhalil')
    txt_file = wf.add_input(txt_file='File')
    metadata = wf.add_input(metadata='File')
    cp = wf.add_input(cp='string')
    size = wf.add_input(size='int?')
        
    txt_files = wf.split_file_chapters(txt_file=txt_file)
        
    analyzed_files = wf.SafarAnalyze(in_files=txt_files, analyzer=analyzer, cp=cp)
    merged_file = wf.merge_safar_xml(in_files=analyzed_files)
    filtered_file = wf.safar_filter_analyses(in_file=merged_file)
    
    out_file = wf.safar_add_metadata_file(in_file=filtered_file, in_file_meta=metadata)
    
    # Output is one xml file
    wf.add_outputs(out_file=out_file)
    
    wf.save('../adhtools/cwl/safar-split-and-analyze-file.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-09 11:56:08.708033. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [9]:
# Split and analyze multiple books
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    
    analyzer = wf.add_input(analyzer='enum', symbols=['Alkhalil', 'BAMA'], default='Alkhalil')
    in_dir = wf.add_input(in_dir='Directory')
    metadata = wf.add_input(metadata='File')
    cp = wf.add_input(cp='string')
    size = wf.add_input(size='int?')  
    
    books = wf.ls(in_dir=in_dir)
    
    safar_output = wf.safar_split_and_analyze_file(analyzer=analyzer, txt_file=books, cp=cp, 
                                                   size=size, metadata=metadata,
                                                   scatter='txt_file', scatter_method='dotproduct')
    
    wf.add_outputs(safar_output=safar_output)
    
    wf.save('../adhtools/cwl/safar-split-and-analyze-dir.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-09 11:57:43.933974. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [5]:
# split and stem file file 
# - remove openiti metadata
# - divide in chapters (separate headings)
# - split based on file size
# - analyze
# - merge xml files
# (- filter analyses)
# - add metadata
# Result: an xml file for a book
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    
    stemmer = wf.add_input(stemmer='enum', 
                           symbols=['KHOJA', 'LIGHT10', 'ISRI', 'MOTAZ', 'TASHAPHYNE'], 
                           default='LIGHT10')
    txt_file = wf.add_input(txt_file='File')
    metadata = wf.add_input(metadata='File')
    cp = wf.add_input(cp='string')
    size = wf.add_input(size='int?')
        
    txt_files = wf.split_file_chapters(txt_file=txt_file)
        
    stemmed_files = wf.SafarStem(in_files=txt_files, stemmer=stemmer, cp=cp)
    merged_file = wf.merge_safar_xml(in_files=stemmed_files)
    filtered_file = merged_file #wf.safar_filter_analyses(in_file=merged_file) # to do: also filter stemmed?
    
    out_file = wf.safar_add_metadata_file(in_file=filtered_file, in_file_meta=metadata)
    
    # Output is one xml file
    wf.add_outputs(out_file=out_file)
    
    wf.save('../adhtools/cwl/safar-split-and-stem-file.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-11 11:03:48.693993. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [11]:
# Split and stem single book (txt file)
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    
    stemmer = wf.add_input(stemmer='enum', 
                           symbols=['KHOJA', 'LIGHT10', 'ISRI', 'MOTAZ', 'TASHAPHYNE'], 
                           default='LIGHT10')
    in_dir = wf.add_input(in_dir='Directory')
    metadata = wf.add_input(metadata='File')
    regex = wf.add_input(regex='string[]', default=['### |', '### ||'])
    cp = wf.add_input(cp='string')
    
    txt_files = wf.ls(in_dir=in_dir)
    out_files = wf.safar_split_and_stem_file(in_file=txt_files, metadata=metadata, stemmer=stemmer, cp=cp,
                                             scatter='in_file', scatter_method='dotproduct')

    wf.add_outputs(out_files=out_files)
    
    wf.save('../adhtools/cwl/safar-split-and-stem-dir.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-09 12:03:36.552569. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [12]:
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    
    in_dir = wf.add_input(in_dir='Directory')
    metadata = wf.add_input(metadata='File')
    
    in_files = wf.ls(in_dir=in_dir)
    out_files = wf.safar_add_metadata_file(in_file=in_files, in_file_meta=metadata,
                                          scatter='in_file', scatter_method='dotproduct')
    
    wf.add_outputs(out_files=out_files)
    wf.save('../adhtools/cwl/add-metadata-dir.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-10-09 12:03:37.083704. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
